In [1]:
import pandas as pd

In [2]:
user_df=pd.read_table('https://static.turi.com/datasets/millionsong/10000.txt',header=None)

In [3]:
user_df.columns=['user_id','song_id','count']
print (user_df.head())

                                    user_id             song_id  count
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B      2
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0      1
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D      1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273      1


In [4]:
print (len(user_df['user_id'].unique()))

76353


In [5]:
song_info_df=pd.read_csv("https://static.turi.com/datasets/millionsong/song_data.csv")

In [6]:
print (song_info_df.head())

              song_id              title  \
0  SOQMMHC12AB0180CB8       Silent Night   
1  SOVFVAK12A8C1350D9        Tanssi vaan   
2  SOGTUKN12AB017F4F1  No One Could Ever   
3  SOBNYVR12A8C13558C      Si Vos Querés   
4  SOHSBXH12A8C13B0DF   Tangle Of Aspens   

                                release       artist_name  year  
0                 Monster Ballads X-Mas  Faster Pussy cat  2003  
1                           Karkuteillä  Karkkiautomaatti  1995  
2                                Butter    Hudson Mohawke  2006  
3                               De Culo       Yerba Brava  2003  
4  Rene Ablaze Presents Winter Sessions        Der Mystic     0  


In [7]:
song_info_df=song_info_df.drop_duplicates(['song_id'])
song_df=user_df.merge(song_info_df,on="song_id")

In [8]:
print (song_df.head())

                                    user_id             song_id  count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1   
1  7c86176941718984fed11b7c0674ff04c029b480  SOAKIMP12A8C130995      1   
2  76235885b32c4e8c82760c340dc54f9b608d7d7e  SOAKIMP12A8C130995      3   
3  250c0fa2a77bc6695046e7c47882ecd85c42d748  SOAKIMP12A8C130995      1   
4  3f73f44560e822344b0fb7c6b463869743eb9860  SOAKIMP12A8C130995      6   

      title             release   artist_name  year  
0  The Cove  Thicker Than Water  Jack Johnson     0  
1  The Cove  Thicker Than Water  Jack Johnson     0  
2  The Cove  Thicker Than Water  Jack Johnson     0  
3  The Cove  Thicker Than Water  Jack Johnson     0  
4  The Cove  Thicker Than Water  Jack Johnson     0  


In [9]:
song_count_df=song_df.groupby(['title']).agg({'count':'count'})
print (song_count_df.head())

                                                    count
title                                                    
#!*@ You Tonight [Featuring R. Kelly] (Explicit...     78
#40                                                   338
& Down                                                373
' Cello Song                                          103
'97 Bonnie & Clyde                                     93


In [10]:
song_count_df=song_count_df.sort_values(by='count', ascending=False)
print (song_count_df.head())

                                count
title                                
Sehr kosmisch                    8277
Undo                             7032
Dog Days Are Over (Radio Edit)   6949
You're The One                   6729
Revelry                          6145


In [11]:
song_popularity_df=song_count_df

In [12]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)

In [13]:
print (train_data.head())

                                          user_id             song_id  count  \
608812   1ed7f8f565fc21c0d7bcfb8c1ef31ff7ffddbf86  SODHKOL12A8C141128      1   
623729   c4e80d31c8368746717f9467a1c2b2edb59bdc1d  SOLWAHM12AB018EBC8      1   
583106   0ec2fa785885b062f3aeb1f4be0104a6678090e2  SOYZLWW12AB0186148      2   
435735   a8b4b7e46ef2568e25656fbbaef1377b9d8add06  SOLCCCU12A58A7FBD9      8   
1361953  28da25926db30d099c77ea72600d612907f221d4  SORCNJV12A58A7A6EF      1   

                           title  \
608812   I'm Not Waiting In Line   
623729                 Overboard   
583106     Dress Me Like a Clown   
435735                 Mardy Bum   
1361953        Land Of Confusion   

                                               release  \
608812                                For Me_ It's You   
623729                                       My Worlds   
583106                             The Dust of Retreat   
435735   Whatever People Say I Am_ That's What I'm Not   
1361953     

In [14]:

import pandas

class MostPopular():
    def __init__(self):
        self.train_data = None
        self.user = None
        self.song = None
        self.pop_rec = None

    def pop_model(self, train_data):
        self.train_data = train_data
        train_df = train_data.groupby('title').agg({'count':'count'}).reset_index()
        train_df = train_df.sort_values(['count', 'title'], ascending = [0,1])
        self.pop_rec = train_df.head(10)

    def rec_pop_model(self):
        return self.pop_rec

In [15]:
pm = MostPopular()
pm.pop_model(train_data)

In [16]:
result_pop=pm.rec_pop_model().reset_index()['title']

In [17]:
from sklearn.externals import joblib
joblib.dump(result_pop, 'mostpop.pkl')

['mostpop.pkl']

In [18]:
pop = joblib.load('mostpop.pkl')

In [26]:
print (len(song_info_df['song_id'].unique()))

999056


In [40]:
import _sqlite3 as sql
con = sql.connect("web app/database.db")

In [41]:
song_info_df.to_sql('songinfo', con, if_exists='replace', index=False)

In [32]:
pd.read_sql('select * from songinfo', con)

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0
5,SOZVAPQ12A8C13B63C,"Symphony No. 1 G minor ""Sinfonie Serieuse""/All...",Berwald: Symphonies Nos. 1/2/3/4,David Montgomery,0
6,SOQVRHI12A6D4FB2D7,We Have Got Love,Strictly The Best Vol. 34,Sasha / Turbulence,0
7,SOEYRFT12AB018936C,2 Da Beat Ch'yall,Da Bomb,Kris Kross,1993
8,SOPMIYT12A6D4F851E,Goodbye,Danny Boy,Joseph Locke,0
9,SOJCFMH12A8C13B0C2,Mama_ mama can't you see ?,March to cadence with the US marines,The Sun Harbor's Chorus-Documentary Recordings,0
